# **Avoided Crossing in 1D Asymmetric Quantum Well**

### [Go back to index](./index.ipynb)



<hr style="height:3px;border:none;color:#333;background-color:#333;" />

<p style="text-align: justify;font-size:15px">
    In quantum physics and quantum chemistry, an avoided crossing is the phenomenon where two 
    eigenvalues of an Hermitian matrix representing a quantum observable and depending on N 
    continuous real parameters cannot become equal in value ("cross") except on a manifold of N-2
    dimensions. Please read the Wikipedia for more infomration about the avoided crossing. 
</p>

<a style="text-align: justify;font-size:15px" href="https://en.wikipedia.org/wiki/Avoided_crossing">https://en.wikipedia.org/wiki/Avoided_crossing</a>

<p style="text-align: justify;font-size:15px">
In this widget, we are going to solve the Schrödinger equation for 1D potential. The formula of the potential is presented below:
</p>

<p style="text-align: center;font-size:15px">
$\large V(x) = x^4 - 0.6x^2 + \mu x$
</p>

<p style="text-align: justify;font-size:15px">
    $x$ is the position. 
    $\mu$ is the potential parameter, which can be tuned by a slider. Move the slider of the $\mu$ 
    parameter from left to right. The figure on the top left shows the well potential (in black) and
    five eigenvlues, which have the lowest eigenvalues. The figure on the top right shows the 
    corresponding eigenvalues. The plot on the bottom shows that how the three lowest eigenvalues 
    change with the potential parameter $\mu$.
    Please check the figure carefully and try to answer the following
    questions:
    <ol style="text-align: left;font-size:15px">
        <li> When will the lowest two eigenvalues close to each other ? </li>
        <li> Will the state order of the eigenvalues change by changing the $\mu$ parameter ? And why ? </li>
        <li> What $\mu$ value make the potential symmetric ? And how are the eigenfunctions and eigenvalues at this $\mu$ value ?</li>
    </ol>
</p>

<p style="text-align: justify;font-size:15px">
   You can pick the eigenvalue and eigenvalue by clicking on the plot. It will highlight the lines.  
</p>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

In [26]:
%matplotlib widget
from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros, concatenate
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt
from math import pi
from numpy import *

colors = ['b', 'r', 'g', 'c', 'm', 'y'] 

def potential(x, mu):
    """Potential function for double trough, mu: Parameter"""
    return x**4 - 0.6*x**2 + mu*x # asymmetrical double trough

def diagonalisierung(hquer, L, N, pot=potential, mu = 0.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         mu: potential function parameter 
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (arry of length N)
         dx: grid space
         V:  Potential at positions x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]               # grid points 
    dx = x[1] - x[0]                              # grid spacing
    V = pot(x, mu)
    z = hquer**2 /2.0/dx**2                       
    h = (diag(V+2.0*z) + diag(-z*ones(N-1), -1)   
                      + diag(-z*ones(N-1), 1) )   #  Hamilton-Matrix

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                  # real part of the eigenvalues
    ind = argsort(ew)                             # Indizes f. sort. Array
    ew = ew[ind]                                  # Sort the ew by ind
    ef = ef[:, ind]                               # Sort the columns
                                                
    ef = ef/sqrt(dx)                              # Correct standardization
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.2, fak= 2.0):
    """Plot der niedrigsten Eigenfunktionen 'ef' im Potential 'V(x)'
       auf Hoehe der Eigenwerte 'ew' in den Plotbereich 'ax'.
       
       Der optionale Parameter 'width' (mit Defaultwert 1)
       gibt die Linienstaerke beim Plot der Eigenfunktionen
       an. 'width' kann auch ein Array von Linienstaerken sein.
       'Emax' (mit Default-Wert V_0/10) legt die Energieobergrenze
       fuer den Plot fest.
       'fak' ist ein Skalierungsfaktor fuer die graphische Darstellung
       der Eigenfunktionen.
    """
    fak = fak/100.0; 

    ax.set_xlim([min(x), max(x)])
    ax.set_ylim([min(V)-0.01, ew[5]-0.02])
    ax2.set_ylim([min(V)-0.01, ew[5]-0.02])
    
    #ax[1].yaxis.set_label_position('right')
    #ax[1].yaxis.tick_right()
    ax.set_xticks([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax.set_xticklabels([-1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5])
    ax.set_xlabel(r'$x/a$', fontsize = 10)
    ax.set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions}$', fontsize = 10)
      
    indmax = sum(ew<=Emax)                       
    
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax.plot(x, fak*abs(ef[:, i])**2+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])
        ax2.plot(x, x*0.0+ew[i], linewidth=width[i]+2.5, color=colors[i%len(colors)])
    
    ax.plot(x, V, c='k', linewidth=1.3)


def plot_zeitentwicklung(ax, ew, ef, x, V, coeff, zeiten, fak): 
    """ Calculate the time evolution of a wave packet.
    """
    
    E0_qm = dot(abs(coeff)**2, ew)                       # qm. Energy expectation
    fak = fak/100.0                                      # Plot scaling factor
    phi_t0 = dot(ef, coeff*exp(-1j*ew*zeiten[0]/hquer))  # Time evolution
                                                         # Plot with line width
                                                         
    phi = dot(ef, coeff*exp(-1j*ew*zeiten[-1]/hquer))
    ax.plot(x, E0_qm+fak*abs(phi)**2, 'r--', linewidth=2.0)


In [27]:
%matplotlib widget
from ipywidgets import FloatSlider, jslink, VBox, HBox, Button
from matplotlib.gridspec import GridSpec

mu = -0.1                                            # Potential parameter
L = 1.5                                              # x Range [-L,L]
N = 200                                              # Number of grid points 
hquer = 0.06                                         # Planck constant
sigma_x = 0.1                                        # The width of the Guassian function
zeiten = linspace(0.0, 10.0, 400)                    # Time


smu = FloatSlider(value = -0.1, min = -0.1, max = 0.1, step = 0.01, description = r'$\mu$: ')
                                                         
fig = plt.figure(figsize=(7,7))
fig.canvas.header_visible = False

gs = GridSpec(11, 6, figure=fig)

ax1 = fig.add_subplot(gs[0:6, 0:5])
ax2 = fig.add_subplot(gs[0:6, 5])
ax3 = fig.add_subplot(gs[7:10, :])

fig.suptitle('Avoided Crossing in 1D Quantum Well', fontsize = 13)

mu1 = []
ew1 = []
ew2 = []
ew3 = []


for i in concatenate((linspace(-0.1, -0.01, 10), linspace(-0.01, 0.01, 20), linspace(0.01, 0.1, 10))):
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = i)
    mu1.append(i)
    ew1.append(ew[0])
    ew2.append(ew[1])
    ew3.append(ew[2])

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = mu)

ax3.plot(mu1, ew1, c='b')
ax3.plot(mu1, ew2, c='r')
ax3.plot(mu1, ew3, c='g')

s3 = ax3.plot(mu, ew[2], 'go', label = str(format(ew[0], '.3f')))
s2 = ax3.plot(mu, ew[1], 'ro', label = str(format(ew[1], '.3f')))
s1 = ax3.plot(mu, ew[0], 'bo', label = str(format(ew[0], '.3f')))

ax3.legend()

ax3.set_xlim([min(mu1), max(mu1)])
ax3.set_ylim([min(ew1), max(ew3)+0.005])

ax3.set_xlabel(r'Potential parameter $\mu$ value', fontsize = 10)
#ax3.set_ylabel(r'The values of the two lowest eigenvalues', fontsize = 10)
ax3.set_xticks([-0.1, -0.05, 0.0, 0.05, 0.1])
ax3.set_xticklabels([-0.1, -0.05, 0.0, 0.05, 0.1])

ax2.yaxis.set_label_position("right")
ax2.yaxis.tick_right()
ax2.get_xaxis().set_visible(False)
ax2.set_ylabel(r'$\rm{\ Eigenvalues}$', fontsize = 10)

plot_eigenfunktionen(ax1, ew, ef, x, V)

sfak = FloatSlider(value = 2, min = 1.0, max = 5.0, step = 1.0, description = r'Zoom factor: ')

def on_mu_change(change):
    global ew, ef, x, dx, V
    ax1.lines = [];
    ax2.lines = [];
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass
    
    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, mu = smu.value)
    plot_eigenfunktionen(ax1, ew, ef, x, V, fak = sfak.value)
    
    ax3.lines.pop(-1)
    ax3.lines.pop(-1)
    ax3.lines.pop(-1)

    ax3.plot(smu.value, ew[0], 'bo', label = str(format(ew[0], '.3f')))
    ax3.plot(smu.value, ew[1], 'ro', label = str(format(ew[1], '.3f')))
    ax3.plot(smu.value, ew[2], 'go', label = str(format(ew[2], '.3f')))
    ax3.legend()

smu.observe(on_mu_change, names = 'value')

def on_press(event):
    global ann, ann1, ixx
    
    ixx = min(enumerate(ew), key = lambda x: abs(x[1]-event.ydata))[0]
    
    for i in range(len(ax2.lines)):
        ax1.lines[i].set_alpha(0.1)
        ax2.lines[i].set_alpha(0.1)
        ax1.lines[i].set_linewidth(1.1)
        
    ax1.lines[ixx].set_alpha(1.0)
    ax2.lines[ixx].set_alpha(1.0)
    ax1.lines[ixx].set_linewidth(2.0)
    
    for i in range(3):
        if i == ixx:
            ax3.lines[i].set_alpha(1.0)
        else:
            ax3.lines[i].set_alpha(0.1)
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass
    
    ann = ax1.annotate(s = 'n = ' + str(ixx+1), xy = (0, ew[ixx]), xytext = (-0.15, ew[ixx]), xycoords = 'data', color='k', size=15)
    ann1 = ax2.annotate(s = str("{:.3f}".format(ew[ixx])), xy = (0, ew[ixx]), xytext = (-1.2, ew[ixx]+0.005), xycoords = 'data', color='k', size=11)

cid = fig.canvas.mpl_connect('button_press_event', on_press)
    
def on_xfak_change(change):
    ax1.lines = [];
    ax2.lines = [];
    plot_eigenfunktionen(ax1, ew, ef, x, V, fak = sfak.value)

sfak.observe(on_xfak_change, names = 'value')

def on_update_click(b):
    for i in ax1.lines:
        i.set_alpha(1.0)
        i.set_linewidth(1.1)
    
    for i in ax2.lines:
        i.set_alpha(1.0)
        
    for i in ax3.lines:
        i.set_alpha(1.0)
    

update = Button(description="Show all")
update.on_click(on_update_click)

display(HBox([smu, sfak]), update)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Show all', style=ButtonStyle())

* **$\mu$:** the potential parameter.
* **Zoom factor:** the zoom factor to show the eigenfunctions. 

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

This work has been done with the support of the EPFL Open Science Fund [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>